This exercise will require you to pull some data from the **Quandl API**. Quandl is currently the most widely used aggregator of financial market data.

*As a first step, register a free account on the http://www.quandl.com website.*

After you register, you will be provided with a unique API key:

In [1]:
API_KEY = 'PBWSMRzsW-m_Hb2HbMfU'

This mini project focuses on equities data from the Frankfurt Stock Exhange (FSE). Analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

Use the *requests* package, which can be easily downloaded using *pip* or *conda*. Documentation for the package can be found here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, do not use any third party Python packages, such as *pandas*, and instead utilize Python's native data structures (https://docs.python.org/3/tutorial/datastructures.html) as well as what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).

In [2]:
# First, import the relevant modules
import requests
import json
import collections

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets'
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
start_date = '2017-01-01'
end_date = '2017-01-02'
test = requests.get(f'{url}/{database_code}/{dataset_code}/data.{return_format}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}')
print(test)
test.text

<Response [200]>


'{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2017-01-01","end_date":"2017-01-02","frequency":"daily","data":[["2017-01-02",34.99,35.94,34.99,35.8,null,44700.0,1590561.0,null,null,null]],"collapse":null,"order":null}}'

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json.loads(test.text)['dataset_data']

{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'start_date': '2017-01-01',
 'end_date': '2017-01-02',
 'frequency': 'daily',
 'data': [['2017-01-02',
   34.99,
   35.94,
   34.99,
   35.8,
   None,
   44700.0,
   1590561.0,
   None,
   None,
   None]],
 'collapse': None,
 'order': None}

These are the tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. What was the median trading volume during this year?

**1)** *Collect data from the Frankfurt Stock Exchange for the ticker AFX_X, for the whole year 2017*

In [5]:
url = 'https://www.quandl.com/api/v3/datasets'
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
start_date = '2017-01-01'
end_date = '2017-12-31'
res = requests.get(f'{url}/{database_code}/{dataset_code}/data.{return_format}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}')

**2)** *Convert the returned JSON object into a Python dictionary*

In [6]:
data = json.loads(res.text)['dataset_data']
data_dict = collections.OrderedDict()
for i in list(range(0,len(data['data']))):
    data_dict[data['data'][i][0]] = {}
    for j in list(range(0,len(data['data'][i]))):
        data_dict[data['data'][i][0]][data['column_names'][j]] = data['data'][i][j]

print('First 2 items:', list(data_dict.items())[0:2])

First 2 items: [('2017-12-29', {'Date': '2017-12-29', 'Open': 51.76, 'High': 51.94, 'Low': 51.45, 'Close': 51.76, 'Change': None, 'Traded Volume': 34640.0, 'Turnover': 1792304.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}), ('2017-12-28', {'Date': '2017-12-28', 'Open': 51.65, 'High': 51.82, 'Low': 51.43, 'Close': 51.6, 'Change': None, 'Traded Volume': 40660.0, 'Turnover': 2099024.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None})]


**3)** *Calculate what the highest and lowest opening prices were for the stock in this period*

In [7]:
max_price = 0
max_price_date = ''
min_price = 100
min_price_date = ''

for i in data_dict.keys():
    opening = data_dict[i]['Open']
    if (opening is None): continue
    if (opening > max_price):
        max_price = opening
        max_price_date = data_dict[i]['Date']
    if (opening < min_price):
        min_price = opening
        min_price_date = data_dict[i]['Date']

print('Highest Opening Price:', max_price, 'on', max_price_date)
print('Lowest Opening Price:', min_price, 'on', min_price_date)

Highest Opening Price: 53.11 on 2017-12-14
Lowest Opening Price: 34.0 on 2017-01-24


**4)** *What was the largest change in any one day (based on High and Low price)*

In [8]:
max_change = 0
max_change_date = ''

for i in data_dict.keys():
    change = data_dict[i]['High'] - data_dict[i]['Low']
    if (change > max_change):
        max_change = change
        max_change_date = data_dict[i]['Date']

print('Largest Change:', max_change, 'on', max_change_date)

Largest Change: 2.8100000000000023 on 2017-05-11


**5)** *What was the largest change between any two days (based on Closing Price)*

In [9]:
max_change_2 = 0
max_change_date_2 = ''
prev_day = data_dict['2017-12-29']['Close']
for i in list(data_dict.keys())[1:]:
    change = data_dict[i]['Close'] - prev_day
    prev_day = data_dict[i]['Close']
    if (change > max_change):
        max_change = change
        max_change_date = data_dict[i]['Date']

print('Largest Change:', max_change, 'on', max_change_date)

Largest Change: 2.8100000000000023 on 2017-05-11


**6)** *What was the average daily trading volume during this year*

In [10]:
trading_volumes = []

for i in data_dict.keys():
    trading_volumes.append(data_dict[i]['Traded Volume'])
    
print('Average Trading Volume:', sum(trading_volumes)/len(trading_volumes))

Average Trading Volume: 89124.33725490196


**7)** *What was the median trading volume during this year*

In [11]:
trading_volumes.sort()
middle = len(trading_volumes)/2
if middle.is_integer():
    print('Median:', trading_volumes[middle])
else:
    middle = int(middle - 0.5)
    median = (trading_volumes[middle]+trading_volumes[middle+1])/2 
    print('Median:', median)

Median: 76443.0
